In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("diffusion_db_unaltered.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 3 columns):
 #   Column       Dtype  
---  ------       -----  
 0   prompt       object 
 1   image_nsfw   float64
 2   prompt_nsfw  float64
dtypes: float64(2), object(1)
memory usage: 45.8+ MB


In [3]:
df_part = df.sample(n=100000, random_state=42)

In [4]:
df_part.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 1828401 to 680282
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   prompt       99968 non-null   object 
 1   image_nsfw   100000 non-null  float64
 2   prompt_nsfw  100000 non-null  float64
dtypes: float64(2), object(1)
memory usage: 3.1+ MB


In [5]:
df[df["prompt_nsfw"] == df["prompt_nsfw"].max()]

,prompt,image_nsfw,prompt_nsfw
1788911,your mama so fat and stupid...,0.075757,0.998352


In [14]:
df.iloc[191144]

prompt         dance first. think later. it's the natural ord...
image_nsfw                                              0.099572
prompt_nsfw                                             0.000374
nsfw                                                       False
Name: 191144, dtype: object

In [7]:
df['nsfw'] = (df['image_nsfw'] >= 0.7) | (df['prompt_nsfw'] >= 0.7)
nsfw_t = df[df['nsfw'] >= 0.7]
nsfw_t

,prompt,image_nsfw,prompt_nsfw,nsfw
64,luxury leather texture,2.000000,0.000914,True
126,"njord viking god of wealth, tom bagshaw",0.928684,0.019328,True
127,"njord viking god of wealth, swedish man, elega...",0.950153,0.006821,True
160,"portrait painting of a west hightland white, u...",2.000000,0.001851,True
172,"only with red, a red lizardman in symbiosis wi...",2.000000,0.002032,True
...,...,...,...,...
1999933,ancient egyptian art john oliver,2.000000,0.001514,True
1999941,"symmetric, a dreamy male android holding a bra...",2.000000,0.003106,True
1999942,"zoomed out, a dark male android in dark blue c...",2.000000,0.009464,True
1999980,"3 d abstract poster design, by kaws, james jea...",0.910557,0.000622,True


In [8]:
nsfw_f = df[df['nsfw'] < 0.7]
nsfw_f

,prompt,image_nsfw,prompt_nsfw,nsfw
0,"a portrait of a female robot made from code, v...",0.554853,0.001621,False
1,a portrait of a female robot made from a cloud...,0.153645,0.000707,False
2,"only memories remain, trending on artstation",0.062496,0.000425,False
3,dream swimming pool with nobody,0.030799,0.003586,False
4,a dog doing weights. epic oil painting.,0.181035,0.030822,False
...,...,...,...,...
1999995,david bowie giving a piggy back ride to ziggy ...,0.152736,0.022676,False
1999996,david bowie giving a piggy back ride to ziggy ...,0.127884,0.014115,False
1999997,funny computer,0.093476,0.000463,False
1999998,hilarious witty computing machine,0.124379,0.002547,False


In [9]:
prompts_nsfw = df[df['prompt_nsfw'] >= 0.7]
prompts_nsfw[['prompt', 'prompt_nsfw']]

,prompt,prompt_nsfw
766,"kpop idol woman dressed in pink, eating a suck...",0.873680
2402,"horrid, abominable, disgusting, vile, revoltin...",0.962162
2403,"horrid, abominable, disgusting, vile, revoltin...",0.990164
2970,die cut sticker of a pair of lips,0.751755
3224,pork belly ramen foodporn,0.733824
...,...,...
1999393,"spiritual meditation into madness, psycho stup...",0.990041
1999394,"spiritual meditation into madness, psycho stup...",0.990041
1999395,"spiritual meditation into madness, psycho stup...",0.990041
1999525,pimp named slickback drag queen,0.735785


In [10]:
pnsfw_m = df[df['prompt_nsfw'] >= 0.99]
pnsfw_m[['prompt', 'prompt_nsfw']]

,prompt,prompt_nsfw
2403,"horrid, abominable, disgusting, vile, revoltin...",0.990164
12458,"spiritual meditation into madness, psycho stup...",0.990041
12459,"spiritual meditation into madness, psycho stup...",0.990041
12460,"spiritual meditation into madness, psycho stup...",0.990041
46197,"trump, extremely fat, stupid hair, doctors exam,",0.994637
...,...,...
1999391,"spiritual meditation into madness, psycho stup...",0.990041
1999392,"spiritual meditation into madness, psycho stup...",0.990041
1999393,"spiritual meditation into madness, psycho stup...",0.990041
1999394,"spiritual meditation into madness, psycho stup...",0.990041


In [15]:
lists = [
    ["attack", "fight", "kill", "murder", "assault", "punch", "stab", "shoot", "beat", "hurt", "injure", "strike", "violence", "aggression", "batter", "wound", "torture", "slaughter", "massacre"],
    ["gun", "knife", "sword", "bomb", "bullet", "firearm", "machete", "axe", "crossbow", "caliber"],
    ["blood", "death", "bloodshed", "massacre", "war", "destruction", "devastation", "bloodbath", "brutality", "cruelty", "terror", "horror", "rage", "hatred", "carnage", "genocide"],
    ["conflict", "uprising", "riot", "warfare", "rebellion", "terrorism", "hostage", "kidnapping", "sacrifice"],
]

In [24]:
def vectorize_prompt(prompt, lists, category_values):
    vector = []
    for i, words in enumerate(lists):
        if any(word in prompt for word in words):
            vector.append(category_values[i])
        else:
            vector.append(0)
    return vector

category_values = [1, 2, 3, 4]

In [27]:
prompt = "The violence attack blood fight."

vector = vectorize_prompt(prompt, lists, category_values)

vector

[1, 0, 3, 0]